The following R commands load the R RDBMS API library, and open a connection to the SQLite GWAS database file

In [1]:
library(DBI)

In [3]:
gwasdb <- dbConnect(RSQLite::SQLite(), "/shared/home/zliu/bds-files/chapter-13-out-of-memory/gwascat.db")

To execute SQL commands, embed your SQL statements as follows

dbGetQuery(gwasdb, 'SELECT * FROM gwascat')

You can directly assign the results of a SQL query to an R data frame class object

In [4]:
df <- dbGetQuery(gwasdb, 'SELECT * FROM gwascat limit 10')

In [5]:
df[3, 'author']

[1] "Ferreira MA"

In [6]:
dbListTables(gwasdb)

[1] "gwascat"

In [21]:
dbListFields(gwasdb, "gwascat")

[1] "id"                       "dbdate"                  
 [3] "pubmedid"                 "author"                  
 [5] "date"                     "journal"                 
 [7] "link"                     "study"                   
 [9] "trait"                    "initial_samplesize"      
[11] "replication_samplesize"   "region"                  
[13] "chrom"                    "position"                
[15] "reported_genes"           "mapped_gene"             
[17] "upstream_gene_id"         "downstream_gene_id"      
[19] "snp_gene_ids"             "upstream_gene_distance"  
[21] "downstream_gene_distance" "strongest_risk_snp"      
[23] "strongest_risk_allele"    "snps"                    
[25] "merged"                   "snp_id_current"          
[27] "context"                  "intergenic"              
[29] "risk_allele_freq"         "pvalue"                  
[31] "pvalue_mlog"              "pvalue_text"             
[33] "or_beta"                  "ci_95"                   
[35] "platform"                 "cnv"

to replace the SQLite dot command ".indices" for which there doesn't appear to be an R API wrapper function, use the following statement which queries the special internal SQLite table "sqlite_master":

In [8]:
dbGetQuery(gwasdb, 'select type, name, tbl_name, sql FROM sqlite_master WHERE type="index" ')

type,name,tbl_name,sql
<chr>,<chr>,<chr>,<chr>


In [18]:
dbGetQuery(gwasdb, 'SELECT trait, chrom, position, strongest_risk_snp, pvalue FROM gwascat LIMIT 5')

trait,chrom,position,strongest_risk_snp,pvalue
<chr>,<chr>,<int>,<chr>,<dbl>
Asthma and hay fever,6,32658824,rs9273373,4e-14
Asthma and hay fever,4,38798089,rs4833095,5e-12
Asthma and hay fever,5,111131801,rs1438673,3e-11
Asthma and hay fever,2,102350089,rs10197862,4e-11
Asthma and hay fever,17,39966427,rs7212938,4e-10


In [19]:
dbGetQuery(gwasdb, 'SELECT trait, chrom, position, strongest_risk_snp, pvalue FROM gwascat ORDER BY trait LIMIT 5')

trait,chrom,position,strongest_risk_snp,pvalue
<chr>,<chr>,<int>,<chr>,<dbl>
&beta;2-Glycoprotein I (&beta;2-GPI) plasma levels,11,34533644,rs836132,5e-08
&beta;2-Glycoprotein I (&beta;2-GPI) plasma levels,11,9090011,rs963167,1e-07
&beta;2-Glycoprotein I (&beta;2-GPI) plasma levels,11,9087740,rs2647528,3e-07
&beta;2-Glycoprotein I (&beta;2-GPI) plasma levels,17,66137608,rs7209395,7e-07
&beta;2-Glycoprotein I (&beta;2-GPI) plasma levels,17,66240200,rs10048158,1e-06


In [20]:
dbGetQuery(gwasdb, 'SELECT trait, chrom, position, strongest_risk_snp, pvalue FROM gwascat WHERE strongest_risk_snp="rs429358"')

trait,chrom,position,strongest_risk_snp,pvalue
<chr>,<chr>,<int>,<chr>,<dbl>
Alzheimer's disease biomarkers,19,44908684,rs429358,5e-14
Alzheimer's disease biomarkers,19,44908684,rs429358,1e-06
Brain imaging,NA,NA,rs429358,NA


In [37]:
dbGetQuery(gwasdb, 'SELECT trait, chrom, position, strongest_risk_snp, pvalue FROM gwascat 
    WHERE position BETWEEN 24000000 AND 25000000 
    AND pvalue IS NOT NULL ORDER BY pvalue LIMIT 5')

trait,chrom,position,strongest_risk_snp,pvalue
<chr>,<chr>,<int>,<chr>,<dbl>
Liver enzyme levels (gamma-glutamyl transferase),22,24603137,rs2073398,1e-109
Gamma glutamyl transpeptidase,22,24594246,rs4820599,7e-53
Myopia (pathological),4,24576474,rs6841898,3e-26
Liver enzyme levels (alkaline phosphatase),6,24491247,rs1883415,6e-26
Body mass index,2,24935139,rs713586,6e-22


In [41]:
dbGetQuery(gwasdb, 'SELECT lower(trait) AS trait, "chr" || chrom || ":" || position AS region FROM gwascat LIMIT 5')

trait,region
<chr>,<chr>
asthma and hay fever,chr6:32658824
asthma and hay fever,chr4:38798089
asthma and hay fever,chr5:111131801
asthma and hay fever,chr2:102350089
asthma and hay fever,chr17:39966427


In [43]:
dbGetQuery(gwasdb, 'SELECT count(pvalue) FROM gwascat')

count(pvalue)
<int>
17279


In [44]:
dbGetQuery(gwasdb, 'SELECT chrom, count(*) FROM gwascat GROUP BY chrom')

chrom,count(*)
<chr>,<int>
NA,70
1,1458
10,930
11,988
12,858
13,432
14,451
15,575
16,704


In [46]:
dbGetQuery(gwasdb, 'SELECT strongest_risk_snp, count(*) AS count FROM gwascat GROUP BY strongest_risk_snp ORDER BY count DESC LIMIT 5')

strongest_risk_snp,count
<chr>,<int>
rs1260326,36
rs4420638,30
rs1800562,28
rs7903146,27
rs964184,25


In [47]:
dbGetQuery(gwasdb, 'SELECT substr(date, 1, 4) AS year FROM gwascat GROUP BY year')

year
<chr>
2005
2006
2007
2008
2009
2010
2011
2012
2013


In [48]:
dbGetQuery(gwasdb, 'SELECT substr(date, 1, 4) AS year, 
    round(avg(pvalue_mlog), 4) AS mean_log_pvalue,
    count(pvalue_mlog) AS n
    FROM gwascat GROUP BY year')

year,mean_log_pvalue,n
<chr>,<dbl>,<int>
2005,6.2474,2
2006,7.2340,8
2007,11.0973,434
2008,11.5054,971
2009,12.6279,1323
2010,13.0641,2528
2011,13.3437,2349
2012,9.6976,4197
2013,10.3643,5406


In [49]:
dbGetQuery(gwasdb, 'SELECT substr(date, 1, 4) AS year, 
    round(avg(pvalue_mlog), 4) AS mean_log_pvalue,
    count(pvalue_mlog) AS n
    FROM gwascat GROUP BY year
    HAVING count(pvalue_mlog) > 10')

year,mean_log_pvalue,n
<chr>,<dbl>,<int>
2007,11.0973,434
2008,11.5054,971
2009,12.6279,1323
2010,13.0641,2528
2011,13.3437,2349
2012,9.6976,4197
2013,10.3643,5406


In [51]:
dbGetQuery(gwasdb, 'SELECT year, avg(num_assoc)
    from (SELECT substr(date, 1, 4) AS year, 
        author,
        count(*) AS num_assoc
        FROM gwascat GROUP BY pubmedid)
    GROUP BY year')

year,avg(num_assoc)
<chr>,<dbl>
2005,1.000000
2006,1.600000
2007,5.878378
2008,7.645669
2009,6.906250
2010,9.216606
2011,7.496815
2012,13.453674
2013,16.605505


In [59]:
dbGetQuery(gwasdb, 'SELECT date, pubmedid, author, strongest_risk_snp FROM gwascat
     WHERE pubmedid="24388013" LIMIT 5')


date,pubmedid,author,strongest_risk_snp
<chr>,<int>,<chr>,<chr>
2013-12-30,24388013,Ferreira MA,rs9273373
2013-12-30,24388013,Ferreira MA,rs4833095
2013-12-30,24388013,Ferreira MA,rs1438673
2013-12-30,24388013,Ferreira MA,rs10197862
2013-12-30,24388013,Ferreira MA,rs7212938


In [81]:
join1 <- dbConnect(RSQLite::SQLite(), "/shared/home/zliu/bds-files/chapter-13-out-of-memory/toy-joins/joins.db")

In [83]:
dbGetQuery(join1,'SELECT * FROM assocs')

Warning message in result_fetch(res@ptr, n = n):
“Column `study_id`: mixed type, first seen values of type integer, coercing other values of type string”


id,study_id,trait,strongest_risk_snp
<int>,<int>,<chr>,<chr>
1,1,"Asthma, hay fever",rs9273373
2,1,"Asthma, hay fever",rs4833095
3,1,"Asthma, hay fever",rs1438673
4,2,Hypertension,rs2820037
5,2,Crohn's disease,rs6596075
6,0,Urate levels,rs12498742


In [84]:
dbGetQuery(join1,'SELECT * FROM studies')

id,pubmedid,year,journal
<int>,<int>,<int>,<chr>
1,24388013,2013,J Allergy
2,17554300,2007,Nature
3,16252231,2005,Am J Hum Genet


In [86]:
dbGetQuery(join1,'SELECT studies.id AS study_id, assocs.id AS assoc_id, trait, year
    FROM assocs INNER JOIN studies ON assocs.study_id = studies.id')

study_id,assoc_id,trait,year
<int>,<int>,<chr>,<int>
1,1,"Asthma, hay fever",2013
1,2,"Asthma, hay fever",2013
1,3,"Asthma, hay fever",2013
2,4,Hypertension,2007
2,5,Crohn's disease,2007


CREATE TABLES

In [92]:
df2 <- list('id integer primary key', 'chrom text', 'start integer', 'end integer', 'strand text', 'name text')

In [100]:
df2 <- data.frame(df2)
df2

X.id.integer.primary.key.,X.chrom.text.,X.start.integer.,X.end.integer.,X.strand.text.,X.name.text.,Freq,Freq.1
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>
id integer primary key,chrom text,start integer,end integer,strand text,name text,1,1


In [101]:
dbDisconnect(gwasdb)